In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = None

In [2]:
# Parameters
query_params = {
    "rolling_window": 1,
    "search_term": "tesla",
    "bq_table_name": "`gdelt-bq.gdeltv2.gkg`",
}
product = {
    "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_n_filter_gdelt_bq.ipynb",
    "data": "/Users/aiujdm2/market_watch/output/data/raw/gdelt-gkg-bq.csv",
}


In [3]:
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from google.cloud import bigquery

import os
import sys
import warnings
from src import utils

# import numpy as np
# import altair as alt
# from altair import datum
# alt.data_transformers.disable_max_rows()
# alt.themes.enable('fivethirtyeight')

warnings.filterwarnings('ignore')

In [4]:
load_dotenv(find_dotenv('market_watch.env'))

True

In [5]:
print(sys.executable)
api_key_file = os.environ['GOOGLE_APPLICATION_CREDENTIALS']
print(api_key_file)

/Users/aiujdm2/.local/share/virtualenvs/market_watch-dtlP-L11/bin/python
/Users/aiujdm2/.ssh/vgaurav-4d0e95d3663a.json


In [6]:
def build_gdelt_query(table_name, search_term, start_date):
    query_string = f"""
        SELECT
          GKGRECORDID,
          DATE,
          SourceCollectionIdentifier,
          DocumentIdentifier,
          V2Locations AS Locations,
          V2Persons AS Persons,
          V2Organizations AS Organizations,
          V2Tone AS Tone
        FROM
          {table_name}
        WHERE
          LOWER(Organizations) LIKE "%{search_term}%"
          AND DATE > {start_date}
    """
    return query_string

In [7]:
def fetch_data(bqclient, query_string):
    df = (
        bqclient.query(query_string)
            .result()
            .to_dataframe(
            # Optionally, explicitly request to use the BigQuery Storage API. As of
            # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
            # API is used by default.
            create_bqstorage_client=True,
        )
    )
    return df

In [8]:
rolling_window = query_params["rolling_window"]
table_name = query_params["bq_table_name"]
search_term = query_params["search_term"]
file_path = product["data"]
start_date = utils.get_start_date(rolling_window)
start_date = utils.gdelt_date_format(start_date)
query = build_gdelt_query(table_name, search_term, start_date)
print(query)


        SELECT
          GKGRECORDID,
          DATE,
          SourceCollectionIdentifier,
          DocumentIdentifier,
          V2Locations AS Locations,
          V2Persons AS Persons,
          V2Organizations AS Organizations,
          V2Tone AS Tone
        FROM
          `gdelt-bq.gdeltv2.gkg`
        WHERE
          LOWER(Organizations) LIKE "%tesla%"
          AND DATE > 20220322164500
    


In [9]:
client = bigquery.Client()
data_df = fetch_data(client, query)
print(f"Processed merged file with {len(data_df)} records")

Processed merged file with 104 records


In [10]:
# Save records
Path(file_path).parent.mkdir(exist_ok=True, parents=True)
data_df.to_csv(file_path)
print(f"Saved file {file_path}")

Saved file /Users/aiujdm2/market_watch/output/data/raw/gdelt-gkg-bq.csv
